In [1]:
from DBEngine import db_engine
from DataService import get_raw, normalize
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset
from NeuralNetwork import NeuralNet, device

In [2]:
data = db_engine.execute(get_raw)

predict_df = pd.DataFrame(data, columns=["id", "lat", "lon", "brightness", "brightness_t31", "radiative_power", "dist_road", "dist_rail", "avg_population"])
print("Available data: " + str(len(predict_df)))

Available data: 121554


In [3]:
predict_df = normalize(predict_df)
predict_np = predict_df.to_numpy()[:, 1:]

prediction_dataset = TensorDataset(
  torch.tensor(predict_np.astype('float64'), dtype=torch.float)
)

predict_df.head(10)

,id,lat,lon,brightness,brightness_t31,radiative_power,dist_road,dist_rail,avg_population
0,8837,0.270542,0.898607,0.440723,0.419776,0.068556,0,0,0.001404
1,106510,0.129975,0.772573,0.21054,0.392816,0.003481,0.167074,0,0.001328
2,99767,0.964595,0.6467,0.360826,0.44812,0.007371,0.003932,0,0.002443
3,23593,0.824422,0.628279,0.189574,0.453002,0.026977,0.063796,0,0.004647
4,50781,0.575205,0.819715,0.285588,0.445497,0.002401,0.560294,0.000004,0.001928
5,113369,0.874773,0.401518,0.142794,0.358496,0.001073,0,0,0.001457
6,9793,0.125429,0.764999,0.054776,0.366584,0.00706,0.344981,0,0.001999
7,10055,0.434811,0.618336,0.149216,0.279875,0.076322,0,0,0.001343
8,50904,0.786244,0.696269,0.227287,0.409574,0.001426,0.884934,0.00001,0.002015
9,99897,0.805068,0.7051,0.297425,0.403017,0.006743,0.00046,0,0.001544


In [4]:
model = NeuralNet().to(device)
model.load('./firms.pt')
print(model)

NeuralNet(
  (f_connected1): Linear(in_features=8, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=2, bias=True)
)


In [5]:
predicted = model.predict(prediction_dataset)
classes = predicted[0].detach().numpy()

In [6]:
predict_df['accident'] = classes[:, 0]
predict_df['attack'] = classes[:, 1]
predict_df['intent'] = np.where(predict_df['accident'] - predict_df['attack'] > 0, 1, 2)

In [7]:
predict_df.sort_values(by=['attack'], ascending=False).head(10)

,id,lat,lon,brightness,brightness_t31,radiative_power,dist_road,dist_rail,avg_population,accident,attack,intent
26045,126941,0.867861,0.448536,0.002015,0.220781,0.000268,0.144016,0.892027,0.276212,-8.341412,8.205842,2
119812,98665,0.907195,0.571614,0.453315,0.287817,0.004483,0.828693,0,0.053629,-8.118444,8.052324,2
96445,95732,0.868831,0.446635,0.009318,0.195497,0.000473,0.499764,0.697946,0.188805,-7.997710,7.942670,2
29052,130185,0.9203,0.509914,0.002141,0.19608,0.000332,0.409809,0.19283,0.054392,-7.697763,7.759478,2
58281,38328,0.981706,0.496054,0.000504,0.203731,0.000381,0.068249,0,0.014878,-7.615116,7.687758,2
65224,31354,0.948898,0.600063,0.005289,0.173273,0.000346,0.019131,0,0.001173,-7.595650,7.660941,2
109858,31271,0.98747,0.499368,0.009759,0.190834,0.000522,0.000103,0,0.013427,-7.589792,7.656676,2
29557,134035,0.920103,0.509764,0.453315,0.301297,0.003897,0.271072,0.217505,0.051715,-7.710611,7.650949,2
90274,89621,0.967721,0.615575,0.009507,0.185442,0.000537,0.056583,0,0.002562,-7.566365,7.627684,2
90259,89591,0.874839,0.459655,0.002141,0.237467,0.000833,0.121418,0.270968,0.15858,-7.463226,7.521459,2


In [8]:
predict_df.sort_values(by=['accident'], ascending=False).head(10)

,id,lat,lon,brightness,brightness_t31,radiative_power,dist_road,dist_rail,avg_population,accident,attack,intent
91614,21859,0.606206,0.289143,0.58358,0.511586,0.930392,0.050997,0,0.002055,11.305595,-11.239069,1
39702,11411,0.081407,0.660995,0.042813,0.340353,0.003248,0.017209,0.982681,0.001789,11.105278,-10.991088,1
33376,8300,0.031711,0.473488,0.159919,0.392087,0.015533,0.074361,0.9834,0.00931,10.670151,-10.672464,1
116397,23203,0.004569,0.506492,0.148524,0.432527,0.012723,0.094533,0.865699,0.006184,10.637230,-10.627471,1
26181,127099,0.014848,0.494425,0.292766,0.435296,0.005676,0.007623,0.978921,0.003478,10.585131,-10.592840,1
45116,18417,0.006337,0.478308,0.136813,0.461964,0.009927,0,0.236241,0.001455,9.795464,-9.719256,1
71014,54776,0.247084,0.546377,0.039854,0.367677,0.000388,0.007432,0.989693,0.002163,9.786254,-9.667788,1
116396,23202,0.004872,0.506321,0.152931,0.433328,0.012652,0.27413,0.944217,0.00568,9.619726,-9.649353,1
69125,32240,0.045111,0.486546,0.215262,0.411396,0.004024,0.002603,0.811463,0.006281,9.555334,-9.544662,1
29292,110778,0.247178,0.546145,0.009885,0.339187,0.000883,0.007937,0.885998,0.002159,9.528104,-9.394150,1


In [9]:
def update(cursor):
  query = "update geo_firms set source=6, accident=%(accident)s, attack=%(attack)s, intent=%(intent)s where id = %(id)s"
  cursor.executemany(query, predict_df.to_dict('records'))

db_engine.execute(update)
db_engine.commit()